## IMPORT DATA

In [13]:
# Imports
import pandas as pd
import sqlite3
import os
import matplotlib.pyplot as plt

# Files config
CSV_TRIGO = 'datasets/trigo-serie-1927-2024.csv'
CSV_MAIZ = 'datasets\maiz-serie-1923-2023.csv'
CSV_SOJA = 'datasets\soja-serie-1941-2023.csv'


# Read CSV
try:
    df_trigo = pd.read_csv(CSV_TRIGO, encoding='latin-1')
except FileNotFoundError:
    print(f"Error: El archivo {CSV_TRIGO} no se encontró.")
    df_trigo = pd.DataFrame()  # Empty DataFrame in case of Error

try:
    df_maiz = pd.read_csv(CSV_MAIZ, encoding='latin-1')
except FileNotFoundError:
    print(f"Error: El archivo {CSV_MAIZ} no se encontró.")
    df_maiz = pd.DataFrame()  # Empty DataFrame in case of Error

try:
    df_soja = pd.read_csv(CSV_SOJA, encoding='latin-1')
except FileNotFoundError:
    print(f"Error: El archivo {CSV_SOJA} no se encontró.")
    df_soja = pd.DataFrame()  # Empty DataFrame in case of Error

## Check and change datatype

In [ ]:
# Paso 1: Inspeccionar los tipos de datos de los DataFrames
print("--- Tipos de datos de df_trigo (Antes) ---")
df_trigo.info()

print("\n--- Tipos de datos de df_maiz (Antes) ---")
df_maiz.info()

print("\n--- Tipos de datos de df_soja (Antes) ---")
df_soja.info()

In [15]:
def ajustar_tipos_datos(df):
    # Ajusta los tipos de datos de las columnas numéricas y de año en el DataFrame.

    # Columnas que deberían ser de tipo numérico
    columnas_numericas = [
        'superficie_sembrada_ha',
        'superficie_cosechada_ha',
        'produccion_tm',
        'rendimiento_kgxha'
    ]

    # Convertir 'anio' a integer (entero)
    # Usamos pd.to_numeric con errors='coerce' para convertir cualquier no-número a NaN
    # Luego rellenamos con 0 y convertimos a entero
    df['anio'] = pd.to_numeric(df['anio'], errors='coerce').fillna(0).astype('int64')

    for col in columnas_numericas:
        # 1. Intentar convertir a tipo numérico (float)
        df[col] = pd.to_numeric(df[col], errors='coerce')

        # 2. Rellenar NaN con 0.
        df[col] = df[col].fillna(0)

        # 3. Convertir a entero (int64)
        df[col] = df[col].astype('int64')

    return df

# Aplicar la función a cada DataFrame
df_trigo = ajustar_tipos_datos(df_trigo)
df_maiz = ajustar_tipos_datos(df_maiz)
df_soja = ajustar_tipos_datos(df_soja)

## Filter Date (1941-2023)

In [16]:
# Definición de los límites del análisis
ANIO_INICIO = 1941
ANIO_FIN = 2023

# Función para aplicar el filtro de años
def filtrar_por_rango(df, nombre_cultivo,anio_inicio, anio_fin):
    """
    Filtra el DataFrame para incluir solo los registros dentro del rango de años especificado.

    Parámetros:
    df (pd.DataFrame): El DataFrame a filtrar.
    anio_inicio (int): Año de inicio del filtro (inclusivo).
    anio_fin (int): Año de fin del filtro (inclusivo).

    Retorna:
    pd.DataFrame: El DataFrame filtrado.
    """
    print(f"\n--- Filtrando {nombre_cultivo} para el rango {anio_inicio}-{anio_fin} ---")
    
    # Crear la máscara booleana con las dos condiciones
    condicion_rango = (df['anio'] >= anio_inicio) & (df['anio'] <= anio_fin)

    # Aplicar el filtro y reasignar el DataFrame
    df_filtrado = df[condicion_rango].copy() # Usamos .copy() para evitar SettingWithCopyWarning

    print(f"Filas originales: {len(df):,}")
    print(f"Filas filtradas: {len(df_filtrado):,}")
    print(f"Años únicos en el resultado: {df_filtrado['anio'].nunique()}")

    return df_filtrado

# Aplicar la función a cada DataFrame
df_trigo_series = filtrar_por_rango(df_trigo,"TRIGO",ANIO_INICIO, ANIO_FIN)
df_maiz_series = filtrar_por_rango(df_maiz,"MAIZ",ANIO_INICIO, ANIO_FIN)
df_soja_series = filtrar_por_rango(df_soja,"SOJA",ANIO_INICIO, ANIO_FIN)


--- Filtrando TRIGO para el rango 1941-2023 ---
Filas originales: 24,966
Filas filtradas: 20,998
Años únicos en el resultado: 83

--- Filtrando MAIZ para el rango 1941-2023 ---
Filas originales: 33,213
Filas filtradas: 28,652
Años únicos en el resultado: 83

--- Filtrando SOJA para el rango 1941-2023 ---
Filas originales: 12,315
Filas filtradas: 12,315
Años únicos en el resultado: 83


## EDA (Análisis Exploratorio de Datos)

- Documentamos y eliminamos las entradas donde la siembra es nula puesto que si este valor es 0, no tiene sentido la cosecha.
- Si la siembre es mayor a cero, pero la cosecha es nula (0), tomaremos la entrada como que la cosecha se ha perdido y es relevante para la estadística.
- Eliminamos entradas con valores negativos.
- Corregimos valores para entradas donde superficie_cosecha > superficie_siempre

In [17]:
def limpiar_por_siembra_y_cosecha(df, nombre_cultivo):
    """
    1. Elimina filas donde la superficie sembrada es 0.
    2. Reporta las filas donde la siembra fue > 0 pero la cosecha fue 0 (pérdida total).

    Parámetros:
    df (pd.DataFrame): El DataFrame a limpiar.
    nombre_cultivo (str): Nombre del cultivo para mensajes informativos.

    Retorna:
    pd.DataFrame: El DataFrame con los registros de siembra nula eliminados.
    """
    print(f"\n=== Limpieza por Siembra y Cosecha para {nombre_cultivo} ===")

    total_filas_original = len(df)
    
    # 1. Eliminar filas donde la siembra es 0
    df_limpio = df[df['superficie_sembrada_ha'] > 0].copy()
    
    filas_eliminadas = total_filas_original - len(df_limpio)
    
    print(f"Filas originales: {total_filas_original:,}")
    print(f"Registros eliminados (siembra = 0 ha): {filas_eliminadas:,}")
    print(f"Filas restantes después de la eliminación: {len(df_limpio):,}")

    # 2. Identificar y reportar filas de PÉRDIDA TOTAL (Siembra > 0, Cosecha = 0)
    condicion_perdida_total = (df_limpio['superficie_cosechada_ha'] == 0)
    filas_perdida_total = df_limpio[condicion_perdida_total].shape[0]

    print(f"Registros conservados de PÉRDIDA TOTAL (Siembra > 0 y Cosecha = 0): {filas_perdida_total:,}")
    
    if filas_perdida_total > 0:
        porcentaje_perdida = (filas_perdida_total / len(df_limpio)) * 100
        print(f"Esto representa el {porcentaje_perdida:.2f}% de los registros válidos.")
    
    # Asegurarse de que en estos casos la producción y el rendimiento sean 0
    # Esto es una doble verificación, ya que en el paso de ajuste de tipos, 
    # los NaNs se convirtieron a 0, lo cual es correcto para una pérdida total.
    df_limpio.loc[condicion_perdida_total, ['produccion_tm', 'rendimiento_kgxha']] = 0

    return df_limpio

# Aplicar la función a cada DataFrame
df_trigo_EDA = limpiar_por_siembra_y_cosecha(df_trigo_series, 'TRIGO')
df_maiz_EDA = limpiar_por_siembra_y_cosecha(df_maiz_series, 'MAIZ')
df_soja_EDA = limpiar_por_siembra_y_cosecha(df_soja_series, 'SOJA')


=== Limpieza por Siembra y Cosecha para TRIGO ===
Filas originales: 20,998
Registros eliminados (siembra = 0 ha): 3
Filas restantes después de la eliminación: 20,995
Registros conservados de PÉRDIDA TOTAL (Siembra > 0 y Cosecha = 0): 800
Esto representa el 3.81% de los registros válidos.

=== Limpieza por Siembra y Cosecha para MAIZ ===
Filas originales: 28,652
Registros eliminados (siembra = 0 ha): 0
Filas restantes después de la eliminación: 28,652
Registros conservados de PÉRDIDA TOTAL (Siembra > 0 y Cosecha = 0): 938
Esto representa el 3.27% de los registros válidos.

=== Limpieza por Siembra y Cosecha para SOJA ===
Filas originales: 12,315
Registros eliminados (siembra = 0 ha): 2
Filas restantes después de la eliminación: 12,313
Registros conservados de PÉRDIDA TOTAL (Siembra > 0 y Cosecha = 0): 169
Esto representa el 1.37% de los registros válidos.


In [18]:
def limpieza_negativos(df, nombre_cultivo):
    print(f"\n=== Limpieza de Valores Negativos para {nombre_cultivo} ===")
    columnas_numericas = [
        'superficie_sembrada_ha',
        'superficie_cosechada_ha',
        'produccion_tm',
        'rendimiento_kgxha'
    ]
    for col in columnas_numericas:
        if (df[col] < 0).any():
            num_negativos = (df[col] < 0).sum()
            print(f"Columna '{col}' tiene {num_negativos} valores negativos. Reemplazando por 0.")
            # df.loc[df[col] < 0, col] = 0
        else:
            print(f"Columna '{col}' no tiene valores negativos.")
    return df

df_maiz_EDA = limpieza_negativos(df_maiz_EDA, 'MAIZ')
df_soja_EDA = limpieza_negativos(df_soja_EDA, 'SOJA')
df_trigo_EDA = limpieza_negativos(df_trigo_EDA, 'TRIGO')


=== Limpieza de Valores Negativos para MAIZ ===
Columna 'superficie_sembrada_ha' no tiene valores negativos.
Columna 'superficie_cosechada_ha' no tiene valores negativos.
Columna 'produccion_tm' no tiene valores negativos.
Columna 'rendimiento_kgxha' no tiene valores negativos.

=== Limpieza de Valores Negativos para SOJA ===
Columna 'superficie_sembrada_ha' no tiene valores negativos.
Columna 'superficie_cosechada_ha' no tiene valores negativos.
Columna 'produccion_tm' no tiene valores negativos.
Columna 'rendimiento_kgxha' no tiene valores negativos.

=== Limpieza de Valores Negativos para TRIGO ===
Columna 'superficie_sembrada_ha' no tiene valores negativos.
Columna 'superficie_cosechada_ha' no tiene valores negativos.
Columna 'produccion_tm' no tiene valores negativos.
Columna 'rendimiento_kgxha' no tiene valores negativos.


In [19]:
def inconsistencias_cosecha_siembra(df, nombre_cultivo):
    print(f"\n=== Verificación de Inconsistencias Cosecha vs Siembra para {nombre_cultivo} ===")
    if ((df['superficie_cosechada_ha'] > df['superficie_sembrada_ha']).any()):
        num_inconsistencias = (df['superficie_cosechada_ha'] > df['superficie_sembrada_ha']).sum()
        print(f"Se encontraron {num_inconsistencias} registros donde la superficie cosechada es mayor que la superficie sembrada.")
        # Corregir las inconsistencias
        df.loc[df['superficie_cosechada_ha'] > df['superficie_sembrada_ha'], 'superficie_cosechada_ha'] = df['superficie_sembrada_ha']
    else:
        print("No se encontraron inconsistencias entre superficie cosechada y superficie sembrada.")
    return df

df_trigo_EDA = inconsistencias_cosecha_siembra(df_trigo_EDA, 'TRIGO')
df_maiz_EDA = inconsistencias_cosecha_siembra(df_maiz_EDA, 'MAIZ')
df_soja_EDA = inconsistencias_cosecha_siembra(df_soja_EDA, 'SOJA')
        


=== Verificación de Inconsistencias Cosecha vs Siembra para TRIGO ===
No se encontraron inconsistencias entre superficie cosechada y superficie sembrada.

=== Verificación de Inconsistencias Cosecha vs Siembra para MAIZ ===
Se encontraron 1 registros donde la superficie cosechada es mayor que la superficie sembrada.

=== Verificación de Inconsistencias Cosecha vs Siembra para SOJA ===
Se encontraron 1 registros donde la superficie cosechada es mayor que la superficie sembrada.


## Unificación

Ahora que los tipos de datos son iguales en todos los datasets, se unifican los dataframes para pasarlos a un nuevo CSV con los tres tipos de cultivos. Para esto utilizaremos la función de pandas **PD.CONCAT()**. Es importante verificar que los tres dataframes tienen las mismas columnas y tipos de datos.

In [20]:


# Lista de los DataFrames a concatenar
dataframes_a_unificar = [df_trigo_EDA, df_maiz_EDA, df_soja_EDA]

# Concatenar todos los DataFrames verticalmente (axis=0, que es el valor por defecto)
# Usamos ignore_index=True para que el índice del nuevo DataFrame sea una secuencia continua (0, 1, 2, ...)
df_granos = pd.concat(dataframes_a_unificar, ignore_index=True)

# Inspección rápida de los DataFrames originales y del unificado
print("\n--- Inspección de los DataFrames originales y del unificado ---")
print(f"Número total de filas en df_trigo: {len(df_trigo_EDA):,}")
print(f"Número total de filas en df_maiz: {len(df_maiz_EDA):,}")
print(f"Número total de filas en df_soja: {len(df_soja_EDA):,}")
print(f"Número total de filas en el DataFrame unificado (df_granos): {len(df_granos):,}")



--- Inspección de los DataFrames originales y del unificado ---
Número total de filas en df_trigo: 20,995
Número total de filas en df_maiz: 28,652
Número total de filas en df_soja: 12,313
Número total de filas en el DataFrame unificado (df_granos): 61,960


### Estandarización de nombres de provincias.

In [21]:
# Contar valor únicos de provincias:
print(f"Valores únicos: {df_granos['provincia_nombre'].nunique()}")
print(df_granos['provincia_nombre'].unique())

Valores únicos: 25
['Buenos Aires' 'Catamarca' 'Córdoba' 'Chaco' 'Chubut' 'Entre Ríos'
 'Jujuy' 'La Pampa' 'La Rioja' 'Mendoza' 'Misiones' 'Neuquén' 'Río Negro'
 'Salta' 'San Juan' 'San Luis' 'Santa Fe' 'Santiago del Estero' 'Tucumán'
 'Corrientes' 'Formosa' 'Santa Cruz' 'Entre RÃ\xados' 'CÃ³rdoba'
 'TucumÃ¡n']


In [22]:
# Estandarizamos los nombres de las provincias que quedaron mal por la codificación.

# Definición del diccionario de reemplazo basado en el output de arriba.
mapeo_correccion_final = {
    'Entre RÃ\xados': 'Entre Ríos',
    'CÃ³rdoba': 'Córdoba',
    'TucumÃ¡n': 'Tucumán'
    ####
}

df_granos['provincia_nombre'] = df_granos['provincia_nombre'].replace(mapeo_correccion_final)

# Verificación de la limpieza
print(f"Valores únicos después de la corrección: {df_granos['provincia_nombre'].nunique()}")
print(df_granos['provincia_nombre'].unique())

Valores únicos después de la corrección: 22
['Buenos Aires' 'Catamarca' 'Córdoba' 'Chaco' 'Chubut' 'Entre Ríos'
 'Jujuy' 'La Pampa' 'La Rioja' 'Mendoza' 'Misiones' 'Neuquén' 'Río Negro'
 'Salta' 'San Juan' 'San Luis' 'Santa Fe' 'Santiago del Estero' 'Tucumán'
 'Corrientes' 'Formosa' 'Santa Cruz']


### Eliminacion de outliers
- Si siembra > 0 y cosecha > 0 pero rinde es 0 -> eliminamos la entrada
- Si el rinde en el grano es mayor a MAX_RINDE_SOJA (trigo o maiz) -> eliminamos la entrada

In [ ]:
# Definición de los límites de rendimiento (kg/ha) (casos muy extremos que puedan afectar estadisticas)
MAX_RINDE_SOJA = 5000 # 50 Quintales
MAX_RINDE_TRIGO = 10000 # 100 Quintales
MAX_RINDE_MAIZ = 13000 # 130 Quintales

def eliminar_outliers_e_inconsistencias(df):
    """
    Aplica dos reglas de limpieza:
    1. Elimina registros donde Siembra > 0, Cosecha > 0, pero Rendimiento = 0.
    2. Elimina registros donde el rendimiento supera un umbral máximo por cultivo.

    Parámetros:
    df (pd.DataFrame): El DataFrame unificado (df_granos).

    Retorna:
    pd.DataFrame: El DataFrame con las filas anómalas eliminadas.
    """
    print("\n--- Aplicando limpieza de Outliers e Inconsistencias ---")
    filas_originales = len(df)
    
    # Inconsistencia Lógica (Si Rendimiento = 0 y debería ser > 0)     
    # Condición de inconsistencia: Siembra > 0, Cosecha > 0, PERO Rendimiento == 0
    condicion_inconsistencia = (
        (df['superficie_sembrada_ha'] > 0) & 
        (df['superficie_cosechada_ha'] > 0) & 
        (df['rendimiento_kgxha'] == 0)
    )
    
    filas_a_eliminar_inconsistencia = df[condicion_inconsistencia].shape[0]
    
    # Filtramos para mantener SOLO las filas que NO cumplen con la inconsistencia
    df_limpio = df[~condicion_inconsistencia].copy()
    
    print(f"Registros eliminados por inconsistencia lógica (Rinde=0, Cosecha>0): {filas_a_eliminar_inconsistencia:,}")

    # Eliminación de Outliers por Rendimiento Extremo

    # Condición para eliminar outliers
    condicion_outlier = (
        (df_limpio['cultivo_nombre'].str.contains('soja', case=False) & (df_limpio['rendimiento_kgxha'] > MAX_RINDE_SOJA)) |
        (df_limpio['cultivo_nombre'].str.contains('trigo', case=False) & (df_limpio['rendimiento_kgxha'] > MAX_RINDE_TRIGO)) |
        (df_limpio['cultivo_nombre'].str.contains('maíz', case=False) & (df_limpio['rendimiento_kgxha'] > MAX_RINDE_MAIZ))
    )
    
    filas_a_eliminar_outliers = df_limpio[condicion_outlier].shape[0]
    
    # Filtramos para mantener SOLO las filas que NO son outliers
    df_limpio_final = df_limpio[~condicion_outlier].copy()
    
    print(f"Registros eliminados por outlier de rendimiento (> Límites): {filas_a_eliminar_outliers:,}")

    filas_eliminadas_total = filas_originales - len(df_limpio_final)
    print(f"Filas totales eliminadas en esta fase: {filas_eliminadas_total:,}")
    print(f"Filas finales en el DataFrame: {len(df_limpio_final):,}")
    
    return df_limpio_final

# Asumiendo que df_granos es tu DataFrame actual (con los nombres de provincia corregidos)
df_granos = eliminar_outliers_e_inconsistencias(df_granos)


--- Aplicando limpieza de Outliers e Inconsistencias ---
Registros eliminados por inconsistencia lógica (Rinde=0, Cosecha>0): 17
Registros eliminados por outlier de rendimiento (> Límites): 7
Filas totales eliminadas en esta fase: 24
Filas finales en el DataFrame: 61,936


In [24]:
# EXPORTAR A NUEVO CSV
# Definición del nombre del archivo de salida
CSV_OUTPUT = 'datasets/granos_argentina_1941_2023.csv'

print(f"\n--- Exportando a CSV: {CSV_OUTPUT} ---")

# Exportar el DataFrame unificado a un nuevo archivo CSV
# index=False evita escribir el índice de Pandas como una columna en el archivo CSV
df_granos.to_csv(CSV_OUTPUT, index=False, encoding='latin-1') # Mismo encoding que los archivos originales

# Inspección rápida del nuevo DataFrame unificado
print(f"Cultivos únicos presentes: {df_granos['cultivo_nombre'].unique()}")


--- Exportando a CSV: datasets/granos_argentina_1941_2023.csv ---
Cultivos únicos presentes: ['trigo' 'maíz' 'soja']
